In [2]:
import os
import os
import shutil
import sys

import numpy as np
from scipy import sparse

import matplotlib.pyplot as plt
#%matplotlib inline

import seaborn as sn
sn.set()

import pandas as pd



import bottleneck as bn
data_root = 'drive/My Drive/reco/data/'
save_dir = data_root+ 'lists'
pro_dir = save_dir

# Model in pytorch
## l2 regularizers are included by default in optimizers??
##will defining the model tf way later

#gen train-drop
#disc combination of drop and generated output pair
#and real and drop for real ones

import torch
import torch.nn as nn
import torch.nn.parallel # not using multi-gpu for the time being
from torch.autograd import Variable

class make_data(nn.Module):
    def __init__(self):
      super(make_data,self).__init__()
    def forward(self,inp,drop_prob=0):
      x1 = torch.nn.functional.normalize(inp,p=2,dim=1) #default 2nd degree normalizer
      x2 = torch.nn.Dropout(p=drop_prob)(x1)
      return x2

class model_gen(nn.Module):
    def __init__(self,in_feature,latent_feature,weight_deca=0.0):
      super(model_gen, self).__init__()
      self.weight_decay = weight_deca
      self.linear1 = nn.Linear(in_feature, latent_feature, bias=True)
      self.linear2 = nn.Linear(latent_feature, in_feature, bias=True)
      self.tanh = nn.Tanh()
    def forward(self,inp,drop_prob=0):
      
      x_lat = self.tanh(self.linear1(inp))

      x_out = self.linear2(x_lat)

      return x_out

    def loss(self,pred,gt):
      pred = nn.functional.log_softmax(pred,dim=-1)
      loss_mat = pred*gt
      loss = -loss_mat.mean(1)
      return loss.mean(0).view(1)

class model_disc(nn.Module):
    def __init__(self,in_feature,latent_feature,weight_deca=0.0):
      super(model_disc, self).__init__()
      self.weight_decay = weight_deca
      self.linear1 = nn.Linear(in_feature, latent_feature, bias=False)
      self.relu = nn.ReLU()
      self.linear2 = nn.Linear(latent_feature, 1, bias=False)
      
    def forward(self,inp,drop_prob=0):
      
      x1 = self.relu((self.linear1(inp)))

      x_out = self.linear2(x1)
      x_out = x_out.mean(0).view(1)

      return x_out


unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))


def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

vad_data_tr, vad_data_te = load_tr_te_data(os.path.join(pro_dir, 'validation_tr.csv'),
                                           os.path.join(pro_dir, 'validation_te.csv'))


N = train_data.shape[0]
idxlist = np.arange(N)

# training batch size
batch_size = 500
batches_per_epoch = int(np.ceil(float(N) / batch_size))

N_vad = vad_data_tr.shape[0]
idxlist_vad = range(N_vad)

# validation batch size (since the entire validation set might not fit into GPU memory)
batch_size_vad = 2000

# the total number of gradient updates for annealing
total_anneal_steps = 200000
# largest annealing parameter
anneal_cap = 0.2

N = train_data.shape[0]
idxlist = np.arange(N)
idxlist

def load_tr_te_data(tp_tr, tp_te):
    #tp_tr = pd.read_csv(csv_file_tr)
    #tp_te = pd.read_csv(csv_file_te)


    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, num_item))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, num_item))
    return data_tr, data_te

vad_data_tr = vad_data_tr
vad_data_te = vad_data_te

import bottleneck as bn
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

# weight initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data,std=0.001)
        print ("got here")
        #m.bias.data.clamp_(-2*0.001, 2*0.001) # this will set the values to 0 but 
        # in tf the values are redrawn : write a function for this if necessary

#call model
num_item = len(unique_sid)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_data = make_data()
model_data.apply(weights_init)
model_data.to(device)

model_disc = model_disc(num_item*2,200,weight_deca=0.01 /500)
#model_disc.apply(weights_init)
model_disc.to(device)

model_gen = model_gen(num_item,200,weight_deca=0.01 /500)
model_gen.apply(weights_init)
model_gen.to(device)

optimizer_gen = torch.optim.Adam(model_gen.parameters(), lr=0.00005, weight_decay=0.0)
optimizer_disc = torch.optim.Adam(model_disc.parameters(), lr=0.00005, weight_decay=0.0)

inputs = torch.FloatTensor(batch_size,num_item )
out_gr = torch.FloatTensor(batch_size,num_item )

inputs.to(device)
out_gr.to(device)

one = torch.FloatTensor([1])
mone = one * -1
one = one.to(device)
mone = mone.to(device)

#for validation data
batch_size_vd = batch_size_vad
no_batches = 10000/batch_size_vd
batch_list_vd = np.arange((no_batches)).astype('int32')
rng = np.random.default_rng()
rng.shuffle(batch_list_vd)





got here
got here


In [3]:

# loop for training

138494/500
batches_per_epoch
batch_list = np.arange((batches_per_epoch-2))
rng = np.random.default_rng()
#batch_list = np.array(batch_list)
#list(range(batch_list))
#print (batch_list)
rng.shuffle(batch_list)

n_epochs = 200
len_epoch = N//batch_size
arr_disc= np.arange(len_epoch)
arr_disc = np.concatenate((arr_disc,arr_disc,arr_disc,arr_disc,arr_disc))
np.random.shuffle(arr_disc)

arr_gen1 = np.arange(len_epoch)
arr_gen1 = np.concatenate((arr_gen1,arr_gen1,arr_gen1,arr_gen1,arr_gen1))
np.random.shuffle(arr_gen1)

arr_gen2 = np.arange(len_epoch)
np.random.shuffle(arr_gen2)
len_epoch


#print (batch_list)
ndcg_vad =[]
from torch.autograd import Variable
ndcgs_vad=[]

for epoch in range(n_epochs):
  rng.shuffle(arr_gen2)
  i = 0

  while (i<len(arr_gen2)):
    #data = training_data[arr_gen2[i]*batch_size:(arr_gen2[i]+1)*batch_size].toarray()
    
    
    #real_cpu = data

    model_gen.zero_grad()
    
    st_idx = arr_gen2[i]*batch_size
    end_idx = min(st_idx + batch_size, N)
    X = train_data[idxlist[st_idx:end_idx]]
    if sparse.isspmatrix(X):
      X = X.toarray()
    X = X.astype('float32')
    input_gen1=X

    inputv=input_gen1
    inputv = Variable(torch.Tensor(inputv)).to(device)
    inputs = Variable(model_data(inputv,drop_prob=0.5).data)
    pred = model_gen(inputs)
    
    #print (inputv.shape)
    
    los = model_gen.loss(pred,inputv) # or input only??
    #los_reg = model.l2_reg().to(device)
    loss_total = los#+los_reg
    #print (los.shape)
    #print ('loss is {0}'.format(los.data[0]) )
    loss_total.backward()
    optimizer_gen.step() 
    i+=1
  print ('epoch is ',epoch)
    #print (i,len(arr_gen2))
##Validate the pre-trained model

epoch is  0
epoch is  1
epoch is  2
epoch is  3
epoch is  4
epoch is  5
epoch is  6
epoch is  7
epoch is  8
epoch is  9
epoch is  10
epoch is  11
epoch is  12
epoch is  13
epoch is  14
epoch is  15
epoch is  16
epoch is  17
epoch is  18
epoch is  19
epoch is  20
epoch is  21
epoch is  22
epoch is  23
epoch is  24
epoch is  25
epoch is  26
epoch is  27
epoch is  28
epoch is  29
epoch is  30
epoch is  31
epoch is  32
epoch is  33
epoch is  34
epoch is  35
epoch is  36
epoch is  37
epoch is  38
epoch is  39
epoch is  40
epoch is  41
epoch is  42
epoch is  43
epoch is  44
epoch is  45
epoch is  46
epoch is  47
epoch is  48
epoch is  49
epoch is  50
epoch is  51
epoch is  52
epoch is  53
epoch is  54
epoch is  55
epoch is  56
epoch is  57
epoch is  58
epoch is  59
epoch is  60
epoch is  61
epoch is  62
epoch is  63
epoch is  64
epoch is  65
epoch is  66
epoch is  67
epoch is  68
epoch is  69
epoch is  70
epoch is  71
epoch is  72
epoch is  73
epoch is  74
epoch is  75
epoch is  76
epoch is 

In [4]:
ndcg_dist = []
ndcg_dist = []
for bnum, st_idx in enumerate(range(0, N_vad, batch_size_vad)):
  end_idx = min(st_idx + batch_size_vad, N_vad)
  X = vad_data_tr[idxlist_vad[st_idx:end_idx]]

  if sparse.isspmatrix(X):
     X = X.toarray()
  X = X.astype('float32')
  #print (j)
  data_vd_tr = X
  data_vd_te = vad_data_te[idxlist_vad[st_idx:end_idx]]#.toarray()
  with torch.no_grad():
    data_vad = (model_data((Variable(torch.Tensor(data_vd_tr)).to(device))))
    pred_val = model_gen(data_vad)
    pred_val = pred_val.cpu().detach().numpy()
    pred_val[data_vd_tr.nonzero()] = -np.inf
    ndcg_dist.append(NDCG_binary_at_k_batch(pred_val, data_vd_te))
  
ndcg_dist = np.concatenate(ndcg_dist)
ndcg_ = ndcg_dist.mean()
print (ndcg_)
  

0.41659939016967257


In [0]:


ndcgs_vad=[]
for epoch in range(n_epochs):
  #data_iter = iter(dataloader)
  np.random.shuffle(idxlist)
  
  i = 0
 
  while i < len_epoch:
    #print (i)
    j =0
    for p in model_disc.parameters():
      p.requires_grad = True
    

    while j < 5 and i<len_epoch:
      
      for p in model_disc.parameters():
        p.data.clamp_(-0.0001, 0.0001)
      st_idx = arr_disc[i*5+j]*batch_size
      end_idx = min(st_idx + batch_size, N)
      X = train_data[idxlist[st_idx:end_idx]]
      if sparse.isspmatrix(X):
        X = X.toarray()
      X = X.astype('float32')
      
      model_disc.zero_grad()
    
      inputs=X
      input_state = Variable(torch.Tensor(inputs)).to(device)
      
      input_drop = model_data(input_state,drop_prob=0.5)
      input_disc = Variable(torch.cat((input_state,input_drop), dim=-1))

      errD_real = model_disc(input_disc)
      #print (mone)
      #print (errD_real.shape)
      errD_real.backward(one)

      # train_with generated data

      st_idx = arr_gen1[i*5+j]*batch_size
      end_idx = min(st_idx + batch_size, N)
      X = train_data[idxlist[st_idx:end_idx]]
      if sparse.isspmatrix(X):
        X = X.toarray()
      X = X.astype('float32')

      input_gen=X
      input_state_gen = Variable(torch.Tensor(input_gen)).to(device)
      input_drop_gen = model_data(input_state_gen,drop_prob=0.5)
      with torch.no_grad():
        out_gen = Variable(model_gen(input_drop_gen).data)
      input_disc_fake = Variable(torch.cat((out_gen,input_drop_gen), dim=-1))
      errD_fake = model_disc(input_disc_fake)
      errD_fake.backward(mone)

      loss_disc = errD_real - errD_fake
      optimizer_disc.step()

      j+=1

    #train the generator

    for p in model_disc.parameters():
      p.requires_grad = False
    model_gen.zero_grad()

    st_idx = arr_gen2[i]*batch_size
    end_idx = min(st_idx + batch_size, N)
    X = train_data[idxlist[st_idx:end_idx]]
    if sparse.isspmatrix(X):
      X = X.toarray()
    X = X.astype('float32')
    input_gen1=X
    input_state_gen1 = Variable(torch.Tensor(input_gen1)).to(device)
    input_drop_gen1 = Variable(model_data(input_state_gen1,drop_prob=0.5).data)
    out_gen1 = model_gen(input_drop_gen1)
    input_disc_fake1 = (torch.cat((out_gen1,input_drop_gen1), dim=-1))
    errD_fake1 = model_disc(input_disc_fake1)
    errD_fake1.backward(one)
    loss_gen = errD_fake1
    optimizer_gen.step() 
    i+=1

  ndcg_dist = []
  ndcg_dist = []
  for bnum, st_idx in enumerate(range(0, N_vad, batch_size_vad)):
    end_idx = min(st_idx + batch_size_vad, N_vad)
    X = vad_data_tr[idxlist_vad[st_idx:end_idx]]

    if sparse.isspmatrix(X):
       X = X.toarray()
    X = X.astype('float32')
    #print (j)
    data_vd_tr = X
    data_vd_te = vad_data_te[idxlist_vad[st_idx:end_idx]]#.toarray()
    with torch.no_grad():
      data_vad = (model_data((Variable(torch.Tensor(data_vd_tr)).to(device))))
      pred_val = model_gen(data_vad)
      pred_val = pred_val.cpu().detach().numpy()
      pred_val[data_vd_tr.nonzero()] = -np.inf
      ndcg_dist.append(NDCG_binary_at_k_batch(pred_val, data_vd_te))
    
  ndcg_dist = np.concatenate(ndcg_dist)
  ndcg_ = ndcg_dist.mean()
  print (ndcg_)
  ndcg_vad.append(ndcg_)




  






0.21512829663820926
0.19795500682493258
0.17994800249617898
0.17196202489108783
0.17235393361780152
0.1770943854273701
0.17058894776473962
0.16651322457128237
0.18461862535381549
0.17421849809272627
0.1816644407080325
0.1850185600695816
0.17851326269247164
0.18605079734858979
0.17930108447948312
0.18532796384985917
0.18980080255811324
0.18251876154542082
0.19645583320334145
0.1908177948022461
0.18291422456241088
0.19155298750315852
0.19147935576667174
0.19218830747927518
0.19719190594356534
0.19836212239497594
0.1969190062846483
0.19436481419995183
0.1907965249343756
0.19092972515196543
0.19593578713588194
0.2004572213619385
0.1982310620105809
0.19510306793221538
0.20178154294147546
0.20179007698894977
0.19528466056840638
0.20287046787954524
0.2011796566935348
0.1995538442190597
0.20262985631431987
0.20187586625887016
0.20284983579177746
0.19992835395799777
0.20702400187586315
0.20086979524971982
0.20560804334398464
0.2083329138438321
0.20263372415590383
0.20801370727818885
0.205563029

In [0]:



import matplotlib.pyplot as plt

#%matplotlib inline
%matplotlib inline
x = np.arange(1,201,1)
plt.plot(x,ndcg_vad,'r-',label='new_approach')

plt.ylabel("Validation NDCG@100")
plt.xlabel("Epochs")
plt.legend()
plt.savefig(save_dir+'/out_map')